In [1]:
from MySQLdb import connect
import witxcraft as wc
from dask.distributed import Client

SINGLESTORE_HOST = "172.17.0.1"
SINGLESTORE_PORT = 3306

SINGLESTORE_URI = "mysql+mysqldb://root:root@{}:{}/demo_big".format(SINGLESTORE_HOST, SINGLESTORE_PORT)

In [2]:
client = Client()
client.cluster

distributed.diskutils - INFO - Found stale lock file and directory '/workspaces/wasi-data-python/examples/dask-worker-space/worker-xuz8gdaa', purging
distributed.diskutils - INFO - Found stale lock file and directory '/workspaces/wasi-data-python/examples/dask-worker-space/worker-wbbixba8', purging
distributed.diskutils - INFO - Found stale lock file and directory '/workspaces/wasi-data-python/examples/dask-worker-space/worker-eyt25p2k', purging
distributed.diskutils - INFO - Found stale lock file and directory '/workspaces/wasi-data-python/examples/dask-worker-space/worker-qfu855_o', purging


LocalCluster(7612d581, 'tcp://127.0.0.1:44117', workers=4, threads=8, memory=15.43 GiB)

In [3]:
import dask.dataframe as dd

df = dd.read_sql_table('comments_small', SINGLESTORE_URI, npartitions=8, index_col='id')
df

/usr/local/lib/python3.9/site-packages/dask/dataframe/io/sql.py:113: SAWarning: Unknown schema content: '  KEY `__UNORDERED` () USING CLUSTERED COLUMNSTORE'
  table = sa.Table(table, m, autoload=True, autoload_with=engine, schema=schema)
/usr/local/lib/python3.9/site-packages/dask/dataframe/io/sql.py:113: SAWarning: Unknown schema content: '  , SHARD KEY () '
  table = sa.Table(table, m, autoload=True, autoload_with=engine, schema=schema)


,text,creation_date,score
npartitions=8,,,
8.509504e+07,object,datetime64[ns],int64
8.933231e+07,...,...,...
...,...,...,...
1.147559e+08,...,...,...
1.189932e+08,...,...,...


In [4]:
len(df)

1000

In [5]:
ml = wc.fromwasmmod('example_wasm.wasm', witx='iface.witx')
ml.sentiment('covid is bad')

compound   -0.542326
positive    0.000000
negative    0.636364
neutral     0.363636
dtype: float64

In [6]:
out = df['text'].apply(ml.sentiment, meta=ml.sentiment.ret_meta)
out.compute()

,compound,positive,negative,neutral
id,,,,
85673569,-0.507974,0.000000,0.190002,0.809998
85559717,-0.367150,0.055316,0.140091,0.804593
85472668,-0.077228,0.000000,0.047619,0.952381
85473382,-0.240590,0.000000,0.000000,1.000000
85599569,0.000000,0.000000,0.000000,1.000000
...,...,...,...,...
115889952,0.968684,0.290068,0.063842,0.646090
115109890,0.202289,0.152542,0.000000,0.847458
115150303,0.401924,0.089855,0.040580,0.869565


In [7]:
from pymysql import connect, cursors
s2 = connect(host=SINGLESTORE_HOST, port=SINGLESTORE_PORT, user='root', passwd='root', db='demo_big', local_infile=True, cursorclass=cursors.DictCursor)
cursor = s2.cursor()

ModuleNotFoundError: No module named 'pymysql'

In [23]:
cursor.execute("select count(*) from comments")
cursor.fetchall()

[{'count(*)': 82037742}]

In [24]:
cursor.execute("""
    create or replace function sentiment_vec(input varchar(256) not null)
    returns table(
        compound double,
        positive double,
        negative double,
        neutral double
    ) as wasm infile '/workspaces/wasi-data-python/examples/example_wasm.wasm';
""")

1

In [25]:
cursor.execute("select * from sentiment_vec('I love wasm')")
cursor.fetchall()

[{'compound': 0.6369499429264264,
  'positive': 0.8076923076923077,
  'negative': 0.0,
  'neutral': 0.1923076923076923}]

In [21]:
cursor.execute("select c.score, c.text, s.* from (select * from comments limit 100) as c, sentiment_vec(c.text) as s")
for row in cursor.fetchall():
    print("-----------------------------------------------------------------------------------------------")
    print(row["text"])
    print("score {} sentiment compound {}".format(row["score"], row["compound"]))
    print("-----------------------------------------------------------------------------------------------")

-----------------------------------------------------------------------------------------------
[Rule of 3/5/0](https://en.cppreference.com/w/cpp/language/rule_of_three) for `printer` is broken.
score 4 sentiment compound -0.4766576055745744
-----------------------------------------------------------------------------------------------
-----------------------------------------------------------------------------------------------
Something like this should work: `allow write: if request.resource.data.message.matches('[a-z0-9-_\*#]+')`. If you're having a hard time making that work, edit your question to show what you tried already.
score 4 sentiment compound 0.4588314677411235
-----------------------------------------------------------------------------------------------
-----------------------------------------------------------------------------------------------
Where is the `test_string` coming from?
score 4 sentiment compound -0.04642568745829288
----------------------------------

In [ ]:
cursor.execute("""
    select
        (score - (score % 20)) as score_bucket, min(s.compound), avg(s.compound), max(s.compound), count(*) as cnt,
        PERCENTILE_CONT(0.9) WITHIN GROUP (ORDER BY s.compound) as "90th percentile",
        PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY s.compound) as "50th percentile",
        PERCENTILE_CONT(0.25) WITHIN GROUP (ORDER BY s.compound) as "25th percentile",
        PERCENTILE_CONT(0.10) WITHIN GROUP (ORDER BY s.compound) as "10th percentile"
    from
        (select * from comments where score > 10) as c,
        sentiment_vec(c.text) as s
    group by 1
    having cnt > 100
    order by 1 asc
""")
rows = cursor.fetchall()

import plotly.figure_factory as ff